## Solutions for ROS Industrial Project

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionStep1">Solution Step 1</a>
* <a href="#SolutionStep2">Solution Step 2</a>
* <a href="#SolutionStep3">Solution Step 3</a>
* <a href="#SolutionStep4">Solution Step 4</a>

## Solution Step 1: Build the URDF <p id="SolutionStep1"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**URDF File: project.urdf.xacro** </p>

In [ ]:
<?xml version="1.0"?>
<robot xmlns:xacro="http://ros.org/wiki/xacro"
       name="ur5" >

  <!-- common stuff -->
  <xacro:include filename="$(find ur_description)/urdf/common.gazebo.xacro" />

  <!-- ur5 -->
  <xacro:include filename="$(find ur_description)/urdf/ur5.urdf.xacro" />

  <!-- arm -->
  <xacro:ur5_robot prefix="" joint_limited="false"/>

  <link name="world" />
  
  <link name="table">
    <visual>
      <geometry>
        <box size="1 1 0.05"/>
      </geometry>
    </visual>
    <collision>
      <geometry>
        <box size="1 1 0.05"/>
      </geometry>
    </collision>
  </link>
  
  <joint name="world_to_table" type="fixed">
  <parent link="world"/>
  <child link="table"/>
  <origin xyz="0 0 0.5" rpy="0 0 0"/>
</joint>

<joint name="table_to_robot" type="fixed">
  <parent link="table"/>
  <child link="base_link"/>
  <origin xyz="0 0 0" rpy="0 0 0"/>
</joint>

</robot>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END URDF File: project.urdf.xacro** </p>

## Solution Step 2: Build the MoveIt package <p id="SolutionStep2"></p>

#### 1. Load the URDF created in the previous step.

<img src="../img/project1.png" width="500" />

#### 2. Create a Virtual Joint.

<img src="../img/project2.png" width="500" />

#### 3. Create the Planning Group.

<img src="../img/project3.png" width="500" />

#### 4. Create a couple of predefined poses for testing.

<img src="../img/project4.png" width="500" />

<img src="../img/project5.png" width="500" />

#### 5. Name your package as project_moveit_config.

## Solution Step 3: Connect the MoveIt package with the simulation <p id="SolutionStep3"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Param File: controllers.yaml** </p>

In [ ]:
controller_list:
  - name: arm_controller
    action_ns: "follow_joint_trajectory"
    type: FollowJointTrajectory
    joints: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint, wrist_3_joint]

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Param File: controllers.yaml** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Param File: joint_names.yaml** </p>

In [ ]:
controller_joint_names: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint, wrist_3_joint]

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Param File: joint_names.yaml** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: ur5_moveit_controller_manager.launch.xml** </p>

In [ ]:
<launch>
  <rosparam file="$(find project_moveit_config)/config/controllers.yaml"/>
  <param name="use_controller_manager" value="false"/>
  <param name="trajectory_execution/execution_duration_monitoring" value="false"/>
  <param name="moveit_controller_manager" value="moveit_simple_controller_manager/MoveItSimpleControllerManager"/>
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: ur5_moveit_controller_manager.launch.xml** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: project_planning_execution.launch** </p>

In [ ]:
<launch>

  <rosparam command="load" file="$(find project_moveit_config)/config/joint_names.yaml"/>

  <include file="$(find project_moveit_config)/launch/planning_context.launch" >
    <arg name="load_robot_description" value="true" />
  </include>

  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="/use_gui" value="false"/>
    <rosparam param="/source_list">[/joint_states]</rosparam>
  </node>

  <include file="$(find project_moveit_config)/launch/move_group.launch">
    <arg name="publish_monitored_planning_scene" value="true" />
  </include>

  <include file="$(find project_moveit_config)/launch/moveit_rviz.launch">
    <arg name="config" value="true"/>
  </include>

</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: project_planning_execution.launch** </p>

## Solution Step 4: Python Script <p id="SolutionStep4"></p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: project_motion.py** </p>

In [ ]:
#! /usr/bin/env python

import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()    
group = moveit_commander.MoveGroupCommander("manipulator")
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path', moveit_msgs.msg.DisplayTrajectory)

group_variable_values = group.get_current_joint_values()

group_variable_values[1] = -1.55
group_variable_values[2] = 1.55
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

group.go(wait=True)

group_variable_values[1] = -1.55
group_variable_values[2] = 0
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

group.go(wait=True)

group_variable_values[1] = -1.55
group_variable_values[2] = 1.55
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

group.go(wait=True)

group_variable_values[1] = 0
group_variable_values[2] = 0
group.set_joint_value_target(group_variable_values)

rospy.sleep(5)

group.go(wait=True)

moveit_commander.roscpp_shutdown()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: project_motion.py** </p>